![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

#  Zero-Shot Named Entity Recognition in Spark NLP

In this notebook, you will find an example of Zero-Shot NER model (`zero_shot_ner_roberta`) that is the first of its kind and can detect any named entities without using any annotated dataset to train a model. 

`ZeroShotNerModel` annotator also allows extracting entities by crafting appropriate prompts to query **any RoBERTa Question Answering model**. 


You can check the model card here: [Models Hub](https://nlp.johnsnowlabs.com/2022/08/29/zero_shot_ner_roberta_en.html)

In [0]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
#nlp.install()

🚨 Your Spark-Healthcare is outdated, installed==4.3.0 but latest version==4.2.4
You can run  jsl.install() to update Spark-Healthcare
🚨 Your Spark-OCR is outdated, installed==4.3.1 but latest version==4.2.1
You can run  jsl.install() to update Spark-OCR


In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL

import os
import json
import string
import numpy as np
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

## Zero-Shot Clinical NER Pipeline

Now we will create a pipeline for Zero-Shot NER model with only `documentAssembler`, `sentenceDetector`, `tokenizer`, `zero_shot_ner` and `ner_converter` stages. As you can see, we don't use any embeddings model, because it is already included in the model. 

Only the thing that you need to do is create meaningful definitions for the entities that you want to extract. For example; we want to detect `PROBLEM`, `DRUG`, `PATIENT_AGE` and  `ADMISSION_DATE` entities, so we created a dictionary with the questions for detecting these entities and the labels that we want to see in the result. Then we provided this dictionary to the model by using `setEntityDefinitions` parameter.

In [0]:
documentAssembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")
    
zero_shot_ner = medical.ZeroShotNerModel.pretrained("zero_shot_ner_roberta", "en", "clinical/models")\
    .setEntityDefinitions(
        {
            "PROBLEM": ["What is the disease?", "What is his symptom?", "What is her disease?", "What is his disease?", 
                        "What is the problem?" ,"What does a patient suffer", 'What was the reason that the patient is admitted to the clinic?'],
            "DRUG": ["Which drug?", "Which is the drug?", "What is the drug?", "Which drug does he use?", "Which drug does she use?", "Which drug do I use?", "Which drug is prescribed for a symptom?"],
            "ADMISSION_DATE": ["When did patient admitted to a clinic?"],
            "PATIENT_AGE": ["How old is the patient?",'What is the gae of the patient?']
        })\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("zero_shot_ner")\
    .setPredictionThreshold(0.1) # default 0.01

ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence", "token", "zero_shot_ner"])\
    .setOutputCol("ner_chunk")\

pipeline = nlp.Pipeline(stages = [
    documentAssembler, 
    sentenceDetector, 
    tokenizer, 
    zero_shot_ner, 
    ner_converter])

zero_shot_ner_model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

zero_shot_ner_roberta download started this may take some time.
[OK!]


In [0]:
zero_shot_ner.extractParamMap()

Out[4]: {Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='engine', doc='Deep Learning engine used for this model'): 'tensorflow',
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='ignoreEntities', doc='List of entities to ignore'): [],
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='predictionThreshold', doc='Minimal confidence score to encode an entity (default is 0.1)'): 0.1,
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='batchSize', doc='Size of every batch'): 8,
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='maxSentenceLength', doc='Max sentence length to process'): 512,
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='caseSensitive', doc='whether to ignore case in tokens for embeddings matching'): True,
 Param(parent='ZeroShotRobertaNer_5d06c0297d21', name='inputCols', doc='previous annotations c

In [0]:
zero_shot_ner.getPredictionThreshold()

Out[5]: 0.1

In [0]:
text_list = ["The doctor pescribed Majezik for my severe headache.",
             "The patient was admitted to the hospital for his colon cancer.",
             "27 years old patient was admitted to clinic on Sep 1st by Dr. X for a right-sided pleural effusion for thoracentesis."
            ]

data = spark.createDataFrame(text_list, F.StringType()).toDF("text")

results = zero_shot_ner_model.transform(data)

In [0]:
results.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|       zero_shot_ner|           ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|The doctor pescri...|[{document, 0, 51...|[{document, 0, 51...|[{token, 0, 2, Th...|[{named_entity, 0...|[{chunk, 21, 27, ...|
|The patient was a...|[{document, 0, 61...|[{document, 0, 61...|[{token, 0, 2, Th...|[{named_entity, 0...|[{chunk, 49, 60, ...|
|27 years old pati...|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 1, 27...|[{named_entity, 0...|[{chunk, 0, 11, 2...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



Lets check the NER model results.

In [0]:
results.selectExpr("explode(zero_shot_ner) AS entity")\
       .select(
           "entity.metadata.word",    
           "entity.result",    
           "entity.metadata.sentence",
           "entity.begin",
           "entity.end",
           "entity.metadata.confidence",
           "entity.metadata.question")\
       .show(100, truncate=False)

+-------------+----------------+--------+-----+---+----------+---------------------------------------------------------------+
|word         |result          |sentence|begin|end|confidence|question                                                       |
+-------------+----------------+--------+-----+---+----------+---------------------------------------------------------------+
|The          |O               |0       |0    |2  |null      |null                                                           |
|doctor       |O               |0       |4    |9  |null      |null                                                           |
|pescribed    |O               |0       |11   |19 |null      |null                                                           |
|Majezik      |B-DRUG          |0       |21   |27 |0.64671516|Which drug is prescribed for a symptom?                        |
|for          |O               |0       |29   |31 |null      |null                                             

In [0]:
results.select(F.explode(F.arrays_zip(results.token.result,
                                      results.zero_shot_ner.result, 
                                      results.zero_shot_ner.metadata,
                                      results.zero_shot_ner.begin, 
                                      results.zero_shot_ner.end)).alias("cols"))\
       .select(F.expr("cols['0']").alias("token"),
               F.expr("cols['1']").alias("ner_label"),
               F.expr("cols['2']['sentence']").alias("sentence"),
               F.expr("cols['3']").alias("begin"),
               F.expr("cols['4']").alias("end"),
               F.expr("cols['2']['confidence']").alias("confidence")).show(50, truncate=100)

+-------------+----------------+--------+-----+---+----------+
|        token|       ner_label|sentence|begin|end|confidence|
+-------------+----------------+--------+-----+---+----------+
|          The|               O|       0|    0|  2|      null|
|       doctor|               O|       0|    4|  9|      null|
|    pescribed|               O|       0|   11| 19|      null|
|      Majezik|          B-DRUG|       0|   21| 27|0.64671516|
|          for|               O|       0|   29| 31|      null|
|           my|               O|       0|   33| 34|      null|
|       severe|       B-PROBLEM|       0|   36| 41|0.55263436|
|     headache|       I-PROBLEM|       0|   43| 50|0.55263436|
|            .|               O|       0|   51| 51|      null|
|          The|               O|       0|    0|  2|      null|
|      patient|               O|       0|    4| 10|      null|
|          was|               O|       0|   12| 14|      null|
|     admitted|               O|       0|   16| 23|    

Now we will check the NER chunks.

In [0]:
results.selectExpr("explode(ner_chunk)").show(100, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------+
|col                                                                                                                                             |
+------------------------------------------------------------------------------------------------------------------------------------------------+
|{chunk, 21, 27, Majezik, {entity -> DRUG, sentence -> 0, chunk -> 0, confidence -> 0.64671516}, []}                                             |
|{chunk, 36, 50, severe headache, {entity -> PROBLEM, sentence -> 0, chunk -> 1, confidence -> 0.55263436}, []}                                  |
|{chunk, 49, 60, colon cancer, {entity -> PROBLEM, sentence -> 0, chunk -> 0, confidence -> 0.88985014}, []}                                     |
|{chunk, 0, 11, 27 years old, {entity -> PATIENT_AGE, sentence -> 0, chunk -> 0, confidence -> 0.6943085}, []}        

In [0]:
results.select(F.explode(F.arrays_zip(results.ner_chunk.result,
                                      results.ner_chunk.metadata)).alias("cols"))\
       .select(F.expr("cols['0']").alias("chunk"),
               F.expr("cols['1']['entity']").alias("ner_label"),
               F.expr("cols['1']['confidence']").alias("confidence")).show(50, truncate=100)

+------------------------------------------------+--------------+----------+
|                                           chunk|     ner_label|confidence|
+------------------------------------------------+--------------+----------+
|                                         Majezik|          DRUG|0.64671516|
|                                 severe headache|       PROBLEM|0.55263436|
|                                    colon cancer|       PROBLEM|0.88985014|
|                                    27 years old|   PATIENT_AGE| 0.6943085|
|                                         Sep 1st|ADMISSION_DATE|0.95646095|
|a right-sided pleural effusion for thoracentesis|       PROBLEM|0.50026613|
+------------------------------------------------+--------------+----------+



## LightPipelines

In [0]:
# fullAnnotate in LightPipeline
print (text_list[-1], "\n")

light_model = nlp.LightPipeline(zero_shot_ner_model)
light_result = light_model.fullAnnotate(text_list[-1])

chunks = []
entities = []
sentence= []
begin = []
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    

df_clinical = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                   'sentence_id':sentence, 'entities':entities})

df_clinical.head(20)

27 years old patient was admitted to clinic on Sep 1st by Dr. X for a right-sided pleural effusion for thoracentesis. 



,chunks,begin,end,sentence_id,entities
0,27 years old,0,11,0,PATIENT_AGE
1,Sep 1st,47,53,0,ADMISSION_DATE
2,a right-sided pleural effusion for thoracentesis,68,115,0,PROBLEM


In [0]:
light_result[0]

Out[13]: {'zero_shot_ner': [Annotation(named_entity, 0, 1, B-PATIENT_AGE, {'sentence': '0', 'word': '27', 'confidence': '0.6943086', 'question': 'How old is the patient?'}),
  Annotation(named_entity, 3, 7, I-PATIENT_AGE, {'sentence': '0', 'word': 'years', 'confidence': '0.6943086', 'question': 'How old is the patient?'}),
  Annotation(named_entity, 9, 11, I-PATIENT_AGE, {'sentence': '0', 'word': 'old', 'confidence': '0.6943086', 'question': 'How old is the patient?'}),
  Annotation(named_entity, 13, 19, O, {'sentence': '0', 'word': 'patient'}),
  Annotation(named_entity, 21, 23, O, {'sentence': '0', 'word': 'was'}),
  Annotation(named_entity, 25, 32, O, {'sentence': '0', 'word': 'admitted'}),
  Annotation(named_entity, 34, 35, O, {'sentence': '0', 'word': 'to'}),
  Annotation(named_entity, 37, 42, O, {'sentence': '0', 'word': 'clinic'}),
  Annotation(named_entity, 44, 45, O, {'sentence': '0', 'word': 'on'}),
  Annotation(named_entity, 47, 49, B-ADMISSION_DATE, {'sentence': '0', 'word'

### NER Visualizer

For saving the visualization result as html, provide `save_path` parameter in the display function.

In [0]:
visualiser = nlp.viz.NerVisualizer()

for i in text_list:

    light_result = light_model.fullAnnotate(i)
    ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)

    # Change color of an entity label
    # visualiser.set_label_colors({'PROBLEM':'#008080', 'DRUG':'#800080', 'PATIENT_AGE':'#808080'})
    # ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', return_html=True)


    # Set label filter
    # ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document',labels=['PROBLEM'], return_html=True)
    
    displayHTML(ner_vis)

The doctor pescribed Majezik DRUG for my severe headache PROBLEM .

The patient was admitted to the hospital for his colon cancer PROBLEM .

27 years old PATIENT_AGE patient was admitted to clinic on Sep 1st ADMISSION_DATE by Dr. X for a right-sided pleural effusion for thoracentesis PROBLEM .

# Save the Model and Load from Disc

Now we will save the Zero-Shot NER model and then we will be able to use this model without definitions. So our model will have the same labels that we defined before.

In [0]:
# save model

zero_shot_ner.write().overwrite().save("dbfs:/databricks/driver/zero_shot_ner_model")

In [0]:
# load from disc and create a new pipeline

zero_shot_ner_local = medical.ZeroShotNerModel.load("dbfs:/databricks/driver/zero_shot_ner_model")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("zero_shot_ner")

ner_converter_local = nlp.NerConverter()\
    .setInputCols(["sentence", "token", "zero_shot_ner"])\
    .setOutputCol("ner_chunk")\

pipeline_local = nlp.Pipeline(stages = [
    documentAssembler, 
    sentenceDetector, 
    tokenizer, 
    zero_shot_ner_local, 
    ner_converter_local])

zero_shot_ner_model_local = pipeline_local.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
# check the results

local_results = zero_shot_ner_model_local.transform(data)

local_results.select(F.explode(F.arrays_zip(local_results.ner_chunk.result,
                                            local_results.ner_chunk.metadata)).alias("cols"))\
             .select(F.expr("cols['0']").alias("chunk"),
                     F.expr("cols['1']['entity']").alias("ner_label"),
                     F.expr("cols['1']['confidence']").alias("confidence")).show(50, truncate=100)

+------------------------------------------------+--------------+----------+
|                                           chunk|     ner_label|confidence|
+------------------------------------------------+--------------+----------+
|                                         Majezik|          DRUG|0.64671516|
|                                 severe headache|       PROBLEM|0.55263436|
|                                    colon cancer|       PROBLEM|0.88985014|
|                                    27 years old|   PATIENT_AGE| 0.6943085|
|                                         Sep 1st|ADMISSION_DATE|0.95646095|
|a right-sided pleural effusion for thoracentesis|       PROBLEM|0.50026613|
+------------------------------------------------+--------------+----------+



# NER Question Generator

`NerQuestionGenerator` annotator helps you build questions on the fly using 2 entities from different labels (preferably a subject and a verb). For example, let's suppose you have an NER model, able to detect `PATIENT`and `ADMISSION` in the following text:

`John Smith was admitted Sep 3rd to Mayo Clinic`
- PATIENT: `John Smith`
- ADMISSION: `was admitted`

You can add the following annotator to construct questions using PATIENT and ADMISSION:

```python
# setEntities1 says which entity from NER goes first in the question
# setEntities2 says which entity from NER goes second in the question
# setQuestionMark to True adds a '?' at the end of the sentence (after entity 2)
# To sum up, the pattern is     [QUESTIONPRONOUN] [ENTITY1] [ENTITY2] [QUESTIONMARK]

qagenerator = NerQuestionGenerator()\
  .setInputCols(["ner_chunk"])\
  .setOutputCol("question")\
  .setQuestionMark(True)\
  .setQuestionPronoun("When")\
  .setStrategyType("Paired")\
  .setEntities1(["PATIENT"])\
  .setEntities2(["ADMISSION"])
```
In the column `question` you will find: `When John Smith was admitted?`. Likewise you could have `Where` or any other question pronoun you may need.

You can use those questions in a QuestionAnsweringModel or ZeroShotNER (any model which requires a question as an input. Let's see the case of QA.

```python
qa = BertForQuestionAnswering.pretrained("bert_qa_spanbert_finetuned_squadv1","en") \
  .setInputCols(["question", "document"]) \
  .setOutputCol("answer") \
  .setCaseSensitive(True)
```
The result will be:

```bash
+--------------------------------------------------------+-----------------------------+
|question                                                |answer                       |
+--------------------------------------------------------+-----------------------------+
|[{document, 0, 25, When John Smith was admitted ? ...}] |[{chunk, 0, 8, Sep 3rd ...}] |
+--------------------------------------------------------+-----------------------------+
```
Strategies:
- Paired: First chunk of Entity 1 will be grouped with first chunk of Entity 2, second with second, third with third, etc (one-vs-one)
- Combined: A more flexible strategy to be used in case the number of chukns in Entity 1 is not aligned with the number of chunks in Entityt 2. The first chunk from Entity 1 will be grouped with all chunks in Entity 2, the second chunk in Entity 1 with again be grouped with all the chunks in Entity 2, etc (one-vs-all).